In [1]:
import tweepy
from tweepy import OAuthHandler
import datetime
import dateutil.parser as parser
import numpy as np
import pandas as pd

In [2]:
def my_profile():
    my_profile = api.me()

    user = my_profile.name
    loc = my_profile.location
    id_me = my_profile.id
    created = my_profile.created_at.date()
    profile_img = my_profile.default_profile_image
    favs = my_profile.favourites_count
    followers = my_profile.followers_count
    friends = my_profile.friends_count
    geo = my_profile.geo_enabled
    profile_img = my_profile.profile_image_url_https
    background_img = my_profile.profile_background_image_url_https
        
    me = {'Id': id_me, 'Name': user, 'Location': loc, 'Created': created, 'Profile image': profile_img, 'Favs count': favs,
          'Followers': followers, 'Following': friends, 'Geolocation': geo, 'Profile image': profile_img,
          'Background image': background_img}
    return me


def hashtags_fn(tweet_i):
    if len(tweet_i.entities.get('hashtags')) != 0:
        hashtags = []
        for hstg in tweet_i.entities.get('hashtags'):
            hashtags.append(hstg.get('text'))
    else:
        hashtags = None
    return hashtags

def user_mentions_fn(tweet_i):
    if len(tweet_i.entities.get('user_mentions')) != 0:
        mentions = []
        for mention in tweet_i.entities.get('user_mentions'):
            mentions.append(mention.get('screen_name'))
    else:
        mentions = None
    return mentions

def urls_fn(tweet_i):
    if len(tweet_i.entities.get('urls')) != 0:
        urls = []
        for url_info in tweet_i.entities.get('urls'):
            urls.append(url_info.get('expanded_url'))
    else:
        urls = None
    return urls

def urls_user(profile):
    if profile.entities.get('url').get('urls') != 0:
        urls = []
        for url_info in profile.entities.get('url').get('urls'):
            urls.append(url_info.get('expanded_url'))
    else:
        urls = None
    return urls

def media_fn(tweet_i):
    if tweet_i.entities.get('media') != None:
        media = []
        for media_info in tweet_i.entities.get('media'):
            media.append(media_info.get('media_url_https'))
    else:
        media = None
    return media

def get_info_tweet(tweet_i):
    tweet_dict = {'Date': tweet_i.created_at,
                  'ID Tweet': tweet_i.id,
                  'User': tweet_i.author._json.get('name'),
                  'Tweet': tweet_i.text,
                  'Language': tweet_i.lang,
                  'Retweet': tweet_i.retweet_count,
                  'Fav': tweet_i.favorite_count,
                  'Reply to id': tweet_i.in_reply_to_user_id,
                  'Reply to name': tweet_i.in_reply_to_screen_name,
                  'Coordinates': tweet_i.coordinates, 
                  'Hastags': hashtags_fn(tweet_i),
                  'URLs': urls_fn(tweet_i),
                  'User mentions': user_mentions_fn(tweet_i),
                  'Images/videos': media_fn(tweet_i)}
    return tweet_dict

def last_n_tweets(n):
    tweets = []
    for tweet in tweepy.Cursor(api.home_timeline).items(n):
        tweets.append(get_info_tweet(tweet))
        
    return pd.DataFrame(tweets).fillna(value = np.nan)


def last_n_tweets_user(user, n):
    tweets = []
    for tweet in tweepy.Cursor(api.user_timeline, id = user).items(n):
        tweets.append(get_info_tweet(tweet))
        
    return pd.DataFrame(tweets).fillna(value = np.nan)

def last_n_tweets_word(word, n):
    tweets = []
    for tweet in tweepy.Cursor(api.search, q = f"\"{word}\"", lang = "es").items(n):
        tweets.append(get_info_tweet(tweet))
        
    return pd.DataFrame(tweets).fillna(value = np.nan)


def last_n_tweets_city(city, n):
    city_data = api.geo_search(query = city, granularity = "city")
    
    for i, v in enumerate(city_data):
        print(i, v.full_name)
        
    num = int(input('Select the number:'))
    
    tweets = []
    for tweet in tweepy.Cursor(api.search, q = "place:%s" % city_data[num].id, lang = "es").items(n):
        tweets.append(get_info_tweet(tweet))
        
    return pd.DataFrame(tweets).fillna(value = np.nan)


def get_profile(user):
    profile = api.get_user(user)

    user = profile.name
    loc = profile.location
    id_me = profile.id
    created = profile.created_at.date()
    profile_img = profile.default_profile_image
    favs = profile.favourites_count
    followers = profile.followers_count
    friends = profile.friends_count
    url_prof_des = urls_user(profile)
    geo = profile.geo_enabled
    profile_img = profile.profile_image_url_https
    background_img = profile.profile_background_image_url_https
        
    profile = {'Id': id_me, 'Name': user, 'Location': loc, 'Created': created, 'Profile image': profile_img, 
               'Favs count': favs, 'Followers': followers, 'Following': friends, 'Geolocation': geo, 
               'Profile image': profile_img, 'Background image': background_img, 'URL profile description': url_prof_des}
    return profile

# Api authentication

First we read our credentials from the file **config_twitter.py** and we conect to twitter through the api.

In [3]:
import config_twitter

auth = OAuthHandler(config_twitter.consumer_key, config_twitter.consumer_secret)
auth.set_access_token(config_twitter.access_token, config_twitter.access_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


# Profile information

## Our profile

We can obtain our profile information.

In [4]:
my_profile()

{'Id': 1363194821490446339,
 'Name': 'Pablo',
 'Location': '',
 'Created': datetime.date(2021, 2, 20),
 'Profile image': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
 'Favs count': 1,
 'Followers': 1,
 'Following': 1,
 'Geolocation': False,
 'Background image': None}

## Other profiles

Also we can get the information from any user profile using its username.

In [5]:
get_profile('Tesla')

{'Id': 13298072,
 'Name': 'Tesla',
 'Location': '',
 'Created': datetime.date(2008, 2, 10),
 'Profile image': 'https://pbs.twimg.com/profile_images/1337607516008501250/6Ggc4S5n_normal.png',
 'Favs count': 3396,
 'Followers': 9897957,
 'Following': 78,
 'Geolocation': True,
 'Background image': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'URL profile description': ['https://www.tesla.com']}

# Tweets

We can make different filters for tweets. For each tweet we obtain the date and time in which it is published, the unique id, the user who publishes it, the language, the number of retweets and favorites, the coordinates from where it is published, if it has been a response to any user and the hastags, urls, mentioned users and images or videos it contains.

## Last tweets

We get the last N tweets posted by users we are following.

In [6]:
last_n_tweets(10)

,Date,ID Tweet,User,Tweet,Language,Retweet,Fav,Reply to id,Reply to name,Coordinates,Hastags,URLs,User mentions,Images/videos
0,2021-05-20 10:41:00,1395328697436033032,Elon Musk,How much is that Doge in the window? https://t...,en,52877,285179,NaN,NaN,NaN,NaN,NaN,NaN,[https://pbs.twimg.com/media/E100DerVEAAfKpp.jpg]
1,2021-05-20 07:54:54,1395286896842985479,Elon Musk,Tesla Model S Plaid delivery event\nJune 3 at ...,en,14164,185267,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-05-19 15:41:10,1395041849438519296,Elon Musk,Credit to our Master of Coin,en,13363,165520,44196397.0,elonmusk,NaN,NaN,NaN,NaN,NaN
3,2021-05-19 14:42:44,1395027147161489412,Elon Musk,Tesla has 💎 🙌,es,58655,454292,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-05-18 13:43:19,1394649803599093762,Elon Musk,Aiming for extreme precision with next gen Mod...,en,4461,91618,44196397.0,elonmusk,NaN,NaN,NaN,NaN,NaN
5,2021-05-18 11:46:18,1394620356674134019,Elon Musk,Giga Berlin suppliers please accelerate!,en,9892,158194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-05-18 09:25:14,1394584856386961411,Elon Musk,https://t.co/13m1KHFMjv,und,19287,278970,NaN,NaN,NaN,NaN,NaN,NaN,[https://pbs.twimg.com/media/E1qPiAaVIAotZ_b.jpg]
7,2021-05-15 22:56:56,1393701964878422017,Elon Musk,RT @SpaceX: Live feed of Starlink mission → ht...,en,10025,0,NaN,NaN,NaN,NaN,"[http://spacex.com/launches, https://twitter.c...",[SpaceX],NaN
8,2021-05-14 16:29:01,1393241952976445440,Elon Musk,RT @Tesla: Configurator now live in Hungary 🇭🇺...,en,5505,0,NaN,NaN,NaN,NaN,NaN,[Tesla],NaN
9,2021-05-14 16:04:57,1393235898767077381,Elon Musk,RT @SpaceX: SpaceX’s fifth high-altitude fligh...,en,20385,0,NaN,NaN,NaN,NaN,NaN,[SpaceX],[https://pbs.twimg.com/ext_tw_video_thumb/1392...


## Last tweets user

We get the last N tweets posted by an user that we specify by its username.

In [7]:
last_n_tweets_user('elonmusk', 10)

,Date,ID Tweet,User,Tweet,Language,Retweet,Fav,Reply to id,Reply to name,Coordinates,Hastags,URLs,User mentions,Images/videos
0,2021-05-21 19:26:00,1395823206901055488,Elon Musk,@cleantechnica Wow!,und,311,3783,17060015,cleantechnica,NaN,NaN,NaN,[cleantechnica],NaN
1,2021-05-21 01:40:10,1395554979851554817,Elon Musk,@teslaownersSV This will actually be able to f...,en,5591,72496,1016059981907386368,teslaownersSV,NaN,NaN,NaN,[teslaownersSV],NaN
2,2021-05-21 01:35:13,1395553735510663169,Elon Musk,@AstroJordy @Tesla We can’t wait to build this...,en,2136,44023,1142527715670519808,AstroJordy,NaN,NaN,[https://twitter.com/i/web/status/139555373551...,"[AstroJordy, Tesla]",NaN
3,2021-05-21 01:31:56,1395552911430619144,Elon Musk,@BTCsessions @TeslaGong @wintonARK Achieving t...,en,1478,9769,2298103730,BTCsessions,NaN,NaN,[https://twitter.com/i/web/status/139555291143...,"[BTCsessions, TeslaGong, wintonARK]",NaN
4,2021-05-20 20:22:14,1395474971485376518,Elon Musk,"@TeslaGong @wintonARK Ideally 0.1kWh, calculat...",en,1185,7701,1008296232261783552,TeslaGong,NaN,NaN,NaN,"[TeslaGong, wintonARK]",NaN
5,2021-05-20 20:20:13,1395474465794838528,Elon Musk,"@BLKMDL3 @TeslaGong @wintonARK Yes, with the S...",en,1118,10386,988975103785365504,BLKMDL3,NaN,NaN,[https://twitter.com/i/web/status/139547446579...,"[BLKMDL3, TeslaGong, wintonARK]",NaN
6,2021-05-20 20:13:36,1395472799020421120,Elon Musk,@TeslaGong @wintonARK Bitcoin hashing (aka min...,en,879,5587,1008296232261783552,TeslaGong,NaN,NaN,[https://twitter.com/i/web/status/139547279902...,"[TeslaGong, wintonARK]",NaN
7,2021-05-20 20:00:53,1395469597680160768,Elon Musk,@wintonARK I agree that this *can* be done ove...,en,897,8799,760345658,wintonARK,NaN,NaN,[https://twitter.com/i/web/status/139546959768...,[wintonARK],NaN
8,2021-05-20 18:00:11,1395439221393870848,Elon Musk,@TeslaOwnersEBay @AustinTeslaClub @Kristennett...,und,465,9077,1175164547746844672,TeslaOwnersEBay,NaN,NaN,[https://twitter.com/i/web/status/139543922139...,"[TeslaOwnersEBay, AustinTeslaClub, Kristennett...",NaN
9,2021-05-20 17:56:35,1395438318095986693,Elon Musk,"@heydave7 Yeah, I haven’t &amp; won’t sell any...",en,12239,49583,29893444,heydave7,NaN,NaN,NaN,[heydave7],NaN


## Last tweets word

We get the last N tweets posted that contain a word.

In [8]:
last_n_tweets_word('covid', 10)

,Date,ID Tweet,User,Tweet,Language,Retweet,Fav,Reply to id,Reply to name,Coordinates,Hastags,URLs,User mentions,Images/videos
0,2021-05-21 19:36:52,1395825943529328645,franilu abrazo a Niall🌟,RT @wester30: El que NO quiera ser vacunado co...,es,183,0,NaN,NaN,NaN,NaN,NaN,[wester30],NaN
1,2021-05-21 19:36:52,1395825941939728386,Josemi,@PDarias83 Miente más que habla. Lo que esta t...,es,0,0,291598146.0,PDarias83,NaN,NaN,[https://twitter.com/i/web/status/139582594193...,[PDarias83],NaN
2,2021-05-21 19:36:52,1395825941553860608,Boca Juniors,RT @metroadelantado: Angileri y Paradela diero...,es,240,0,NaN,NaN,NaN,NaN,NaN,[metroadelantado],[https://pbs.twimg.com/media/E17Qd32XMAAdNb4.jpg]
3,2021-05-21 19:36:51,1395825940006047752,naya ミ☆,RT @RivassFabi: Que bajon lo de la reserva de ...,es,1,0,NaN,NaN,NaN,NaN,NaN,[RivassFabi],NaN
4,2021-05-21 19:36:51,1395825938340974596,La Niki,RT @coticordeiro: covid me estás cagando la vida,es,2163,0,NaN,NaN,NaN,NaN,NaN,[coticordeiro],NaN
5,2021-05-21 19:36:51,1395825936881405956,Maria Jose A. Salom,RT @AlbertoRodNews: LO ÚLTIMO | Emergencia hum...,es,22,0,NaN,NaN,NaN,NaN,NaN,[AlbertoRodNews],NaN
6,2021-05-21 19:36:51,1395825936134717445,Julián Roqué,RT @arilijalad: El FMI propone destinar 50.000...,es,315,0,NaN,NaN,NaN,NaN,NaN,[arilijalad],NaN
7,2021-05-21 19:36:50,1395825935023316994,O'Brien Menéndez Pilar,RT @TReporta: Diputados independientes reaccio...,es,65,0,NaN,NaN,NaN,NaN,NaN,"[TReporta, asambleapa]",NaN
8,2021-05-21 19:36:50,1395825933626585094,Htd0cs,@mccoam Entiendo corazón tengo otra pregunta y...,es,0,0,243630697.0,mccoam,NaN,NaN,[https://twitter.com/i/web/status/139582593362...,[mccoam],NaN
9,2021-05-21 19:36:50,1395825932095696900,piiiipi B,RT @coticordeiro: covid me estás cagando la vida,es,2163,0,NaN,NaN,NaN,NaN,NaN,[coticordeiro],NaN


## Last tweets location

We get the last N tweets posted from a location.

In [9]:
last_n_tweets_city('Valencia', 10)

0 Valencia, Spain
1 Valencia West, AZ
2 Community of Valencia, Spain
3 Valencia, Comunidad Valenciana
4 Valencia, PA
5 Valencia, NM
6 Valencia del Ventoso, España
7 Valenciano Abajo, Puerto Rico
8 Valencia, Colombia
9 Valencia, Venezuela
10 City of Valencia, Northern Mindanao
11 Valencia del Mombuey, España
12 Valencia de Alcántara, España
13 Valenciano Arriba, Puerto Rico
14 Valencia, Ecuador
15 Nueva Valencia, Western Visayas
16 Valencia de Don Juan, España
17 Valencia de las Torres, España
18 Valencia, Central Visayas
19 Valencia, Central Visayas
Select the number:0


,Date,ID Tweet,User,Tweet,Language,Retweet,Fav,Reply to id,Reply to name,Coordinates,Hastags,URLs,User mentions,Images/videos
0,2021-05-21 19:34:17,1395825290874589190,kiko🪐,tengo miedo d abrir mi isla del animal crossin...,es,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-21 19:31:09,1395824502488158209,Romancito 🌈,@Kiaraapaez Las cosas como sooon,es,0,1,1.219063e+18,Kiaraapaez,NaN,NaN,NaN,[Kiaraapaez],NaN
2,2021-05-21 19:30:58,1395824458229821443,Romancito 🌈,@abrilmorinigoo Creo que son las harinas jajaj...,es,0,0,1.151681e+18,abrilmorinigoo,NaN,NaN,NaN,[abrilmorinigoo],NaN
3,2021-05-21 19:30:48,1395824413984137224,Romancito 🌈,@sweetlondraa Jajajajaja juro que no miento,es,0,1,1.268265e+18,sweetlondraa,NaN,NaN,NaN,[sweetlondraa],NaN
4,2021-05-21 19:30:32,1395824349286998019,Romancito 🌈,@mileezaccari Después me contás jajajajaja,es,0,0,8.362778e+17,mileezaccari,NaN,NaN,NaN,[mileezaccari],NaN
5,2021-05-21 19:28:29,1395823832875905029,colorada venga al mote🚬🍻🔪,@Idolo79 @Caravana750 Imagínate que los partid...,es,0,0,3.086438e+09,Idolo79,NaN,NaN,NaN,"[Idolo79, Caravana750]",NaN
6,2021-05-21 19:27:46,1395823650197102599,𝐆𝐚𝐛𝐫𝐢𝐞𝐥 𝐆𝐚𝐫𝐜𝐢𝐚°,¿Os gusta? 🤪 https://t.co/iCm5CAck4M,es,0,2,NaN,NaN,NaN,NaN,NaN,NaN,[https://pbs.twimg.com/media/E172NUlWUAQUL9v.jpg]
7,2021-05-21 19:27:13,1395823513580232708,Gabriel Doménech,@DerechoEspana Supongo que si te dejamos fuera...,es,0,1,1.001478e+18,DerechoEspana,NaN,NaN,NaN,[DerechoEspana],NaN
8,2021-05-21 19:25:39,1395823119567396870,Shiba Biosca 🦖🦕,Ya me duplica el perro hoooodll https://t.co/O...,es,0,0,NaN,NaN,NaN,NaN,[https://twitter.com/costaico/status/139582104...,NaN,NaN
9,2021-05-21 19:24:12,1395822755094929409,Jaume Reservoir,Afónico ya de insultar a Negredo.,es,0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
